In [ ]:
fitness = lab9_lib.make_problem(5)
for n in range(20):
    ind = choices([0, 1], k=100)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

In [ ]:
def parent_selection(
    population: list[Individual], tournament_size: int
) -> Individual:
    # we also want to take the last best one.
    parents_idx = np.random.choice(
        range(len(population)), size=tournament_size, replace=False
    )
    parents = [population[idx] for idx in parents_idx]
    return max(parents, key=lambda i: i.fitness)


def uniform_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    assert len(ind1.genotype) == len(ind2.genotype)p1 = ind1.fitness / (ind1.fitness + ind2.fitness)
    mask = np.random.choice([True, False], size=LOCI, p=[p1, 1 - p1])
    gene = np.where(mask, ind1.genotype, ind2.genotype)
    new_ind = Individual()
    new_ind.genotype = gene.tolist()
    return new_ind
    

In [ ]:
from random import choices, random, randint, shuffle
import numpy as np
from copy import deepcopy
from IPython.display import clear_output
import matplotlib.pyplot as plt

import lab9_lib

In [ ]:
class new_generation:
    def __init__(self, mutation_prob: float, bit_to_1:float, bit_change: float):
        self.prob_to_set_1 = bit_to_1
        self.bit_change = bit_change
        self.better_with_0_1 = [0,0]
    
    def mutate(self, parent):
        new_offspring = deepcopy(parent)
        bit_val = 1 if random() <  self.prob_to_set_1 else 0
        for i in range(len(parent)):
            if random() < self.bit_change:
                new_offspring.genotype[i] = bit_val
        new_offspring.fitness = fitness(new_offspring.genotype)
        if new_offspring.fitness > parent.fitness:
            self.better_with_0_1[bit_val]+=1
        return new_offspring
    
    def adjust_prob(self):
        if self.better_with_0_1[1] > self.better_with_0_1[0]:
            self.prob_to_set_1 += 0.1
        else:
            self.prob_to_set_1 -= 0.1
        

    def __call__(self, population: list[Individual], offspring_size, tournament_size) -> list[Individual]:
        offsprings = []
        for _ in range(offspring_size):
            parent1 = parent_selection(population, tournament_size)
            parent2 = parent_selection(population, tournament_size)
            offspring = uniform_cut_xover(parent1, parent2)
            offspring = self.mutate(offspring) if random() < random() < self.mutation_prob else offspring
            offsprings.append(offspring)
        return offsprings